# Exploratory data analysis

In [2]:
import pandas as pd

In [3]:
cases = pd.read_csv("Validated_V0_Desriptions.csv")

In [4]:
cases.head()

,File Names,Attack Link,Exploit Types,Minimum Time,train/test splits,Number of Labeled Transactions,Number of Transactions,Validated,Sum of Stolen Amount
0,Bo_Shen001,https://defiyield.app/rekt-database/bo_shen,Access Control,30/11/2016 12:41,Train,2,144,Yes,4.013972e+07
1,Bancor001,https://www.apriorit.com/dev-blog/554-bancor-e...,Access Control,10/06/2017 14:27,Train,2,580,Yes,3.126104e+06
2,Stazie001,https://coinlive.me/how-did-the-founder-of-nft...,Access Control,26/01/2018 20:36,Train,3,1710,Yes,1.999709e+06
3,Taylor001,https://www.zdnet.com/article/all-of-cryptocur...,Access Control,23/03/2018 15:59,Train,1,7,Yes,1.715527e+06
4,MerkleTreeVentures001,NaN,Access Control,10/05/2018 15:25,Train,2,21,Yes,8.460593e+04


In [5]:
cases.groupby(["Exploit Types", "train/test splits"])["File Names"].count()

Exploit Types       train/test splits
Access Control      Test                  6
                    Train                54
                    Validation           11
Flash Loan Attack   Test                  2
                    Train                18
                    Validation            5
Phishing            Test                  2
                    Train                16
                    Validation            5
Price Manupilation  Train                 5
                    Validation            1
Rug pull            Test                  4
                    Train                25
                    Validation            6
Name: File Names, dtype: int64

In [6]:
cases.groupby(["Exploit Types", "train/test splits"])[
    "Number of Labeled Transactions"
].mean()

Exploit Types       train/test splits
Access Control      Test                 24.666667
                    Train                24.722222
                    Validation           12.363636
Flash Loan Attack   Test                 14.500000
                    Train                 1.055556
                    Validation           26.400000
Phishing            Test                  4.500000
                    Train                 1.625000
                    Validation            4.800000
Price Manupilation  Train                 1.000000
                    Validation            5.000000
Rug pull            Test                  1.750000
                    Train                12.560000
                    Validation            4.000000
Name: Number of Labeled Transactions, dtype: float64

In [7]:
data = pd.read_csv("data/raw_data.csv", parse_dates=["block_time"])

/tmp/ipykernel_401/1695098993.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/raw_data.csv", parse_dates=["block_time"])


In [8]:
case_split = cases.loc[:, ["File Names", "train/test splits"]]
case_split.columns = ["file_name", "experiment_part"]
case_split["file_name"] += ".csv"

In [9]:
data = data.merge(case_split)

In [10]:
data.groupby("experiment_part")["label"].value_counts()

experiment_part  label
Test             0         95353
                 1           193
Train            0        553327
                 1          1699
Validation       0        143502
                 1           321
Name: label, dtype: int64

In [11]:
data.groupby(["experiment_part", "label"])["transaction_id"].nunique()

experiment_part  label
Test             0         41105
                 1           170
Train            0        515418
                 1          1652
Validation       0         98520
                 1           195
Name: transaction_id, dtype: int64

## Look for transactions that shoud be excluded from the dataset

Transactions with very low dollar amounts.

In [12]:
g = data.groupby("transaction_id")

In [13]:
tx_data = pd.DataFrame()

In [14]:
tx_data["label"] = g["label"].max()
tx_data["amount_usd"] = g["amount_usd"].sum()

In [38]:
low_dollar_tx = tx_data.loc[
    (tx_data["label"] == 1) & (tx_data["amount_usd"] < 1), :
].index
len(low_dollar_tx)

268

In [30]:
target_data = data.loc[data["label"] == 1, :].copy()

In [31]:
target_data["low_dollar_amt"] = target_data["transaction_id"].isin(low_dollar_tx)

In [32]:
target_data.groupby("file_name")["low_dollar_amt"].mean().sort_values(
    ascending=False
).head(10)

file_name
Unidark001.csv      0.893130
Kucoin001.csv       0.379487
CoinSWOP001.csv     0.333333
Kucoin002.csv       0.316940
Lendfme001.csv      0.160000
BasketDAO001.csv    0.142857
Nomad001.csv        0.090909
Cryptopia001.csv    0.040000
Liquid002.csv       0.037736
Bitmart001.csv      0.012048
Name: low_dollar_amt, dtype: float64

In [34]:
target_data.loc[
    target_data["file_name"] == "Unidark001.csv", ["transaction_id", "amount_usd"]
]

,transaction_id,amount_usd
522777,0x159245dcf1f0214fb365e9d1bb0770ca036007f39463...,65.551136
522830,0xa6772f13dfc1aac537987513c9447f6ba2f5d3c10d3d...,311.945828
522841,0xb07d5312af46b925d44935f4a46f04670cabd47766a5...,1454.561452
522868,0xc32b64cbff6650812e77266aebc56dd0d264328a73a2...,2025.545079
522895,0x2fadfe543b10f86446e45ed5c40aaf5422f9bc23a3cf...,3453.286374
...,...,...
524822,0x24d5b928a936ab4c84232b5bc25a8eb14af113ece37d...,0.000000
524823,0x201b8e657c75171aaa126859f6c3304722f0713359fb...,0.000000
524824,0x1c1a6ed586e95ed1462d7d004e33a3ced4abdec3119a...,0.000000
524825,0xcc8ec6c1d38c165a0f8d8331ed38a8b21eea4d75406e...,0.000000


In [35]:
target_data.loc[
    target_data["file_name"] == "Kucoin001.csv", ["transaction_id", "amount_usd"]
]

,transaction_id,amount_usd
301451,0x232211d5be7cee21d7719e67e473c29d0b69e508bc8b...,9.213716e+05
301452,0x3c62b8e9f646c431f7bb829e928ab85367adf6ed9d2c...,4.230000e-13
301453,0x56b552ac390fc33a5dd83ceb26d0fc01feff5752c071...,NaN
301454,0xfa130e3e78c97b8238303149b5bbf9ad8c93c3e06594...,8.720000e-13
301455,0x97e7835d30fcc383d9286db0683c1e8fa4492a6f3205...,7.150000e-13
...,...,...
301737,0x6edcf53bc46ee5fbcd5d1895193dcebbfd2aad526725...,0.000000e+00
301740,0x91075c0508834ca1874ef1a485921fe94f64e8dfc4af...,1.349524e+06
301741,0x4f956345abd0d9f3a3ee411291840d2a5d69dfa80cd0...,NaN
301742,0x087061a0e062bd436be8f3b08cac795725637c897c85...,4.807435e+06
